# Minimum example of weird snow netCDF behaviour

In [1]:
from xarray import open_dataset, open_mfdataset
from glob import glob
from os import mkdir
from os.path import join, exists, basename
from tarfile import open as open_tar
from matplotlib import pyplot as plt
from numpy import count_nonzero, isnan, mean
from numpy.ma import count_masked
import netCDF4
import numpy as np
import pandas as pd
import cartopy.crs as ccrs

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.04/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38669 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/38669/status,
Dashboard: /proxy/38669/status,Workers: 7
Total threads: 14,Total memory: 63.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38529,Workers: 7
Dashboard: /proxy/38669/status,Total threads: 14
Started: Just now,Total memory: 63.00 GiB
Comm: tcp://127.0.0.1:38453,Total threads: 2
Dashboard: /proxy/38281/status,Memory: 9.00 GiB
Nanny: tcp://127.0.0.1:41373,


In [5]:
# File paths

# Unpacked level zero data
data_dir = "/scratch/hd50/jt4085/cylc-run/pp_BARRA-R2_SNW/run1/share/cycle/19930301T0000Z/nc/SLV3H"
file1 = "snow_amt_lnd-barra_r2-hres-199303200900-199303201500.nc"
file2 = "snow_amt_lnd-barra_r2-hres-199303201500-199303202100.nc"

filepath1 = join(data_dir, file1)
filepath2 = join(data_dir, file2)

files = [filepath1, filepath2]

In [6]:
# Open them both with xarray
# This is the same arguments as used in productive
ds = open_mfdataset(files,
                    chunks={'time': 'auto'},
                    combine="nested",
                    concat_dim="time",
                    parallel=True,
                    coords='minimal',
                    compat='override')

In [7]:
# Save this dataset to a file
test_path = "/g/data/hd50/jt4085/qc_issues/224/test_combined.nc"

ds.to_netcdf(test_path)

In [8]:
# Now open the new file with xarray
ds_combined = open_dataset(test_path)

In [15]:
# Check the minima
print(ds['snow_amt_lnd'].min().compute().values)
print(ds_combined['snow_amt_lnd'].min().compute().values)

-0.0028076171875
-3.94189453125
